## Evalute the entanglement of real-valued quantum states

Negativity is a measure of quantum entanglement which is easy to compute if one already obtains the density matrix $\rho$. It is a measure deriving from the PPT criterion for separability [1]. Directly compute negativity on quantum hardware is challenging since the partial transpose operation involved is non-physical, which could generate non-postive eigenvalues and hence not a CPTP map. 

The logarithmic negativity is defined as
$$
E_{\text{N}}(\rho) \equiv \log _2\left\|\rho^{\Gamma_A}\right\|_1,
\tag{1}
$$
where $\rho^{\Gamma_A}$ is the partial transpose of $\rho$ with respect to subsystem $A$ and $\|X\|_1=\operatorname{Tr}|X|=\operatorname{Tr} \sqrt{X^{\dagger} X}$ is the trace norm or the sum of the singular values of the operator  $X$.


### Benchmark the entanglement meausre

It is known that $E_{\text{N}}(\rho)=0$ for separable states $\rho = \rho_A \otimes \rho_B$. 

In [3]:
import numpy as np
import hadamard_random_forest as hrf

num_sysA = 2
num_sysB = 3
num_qubits = num_sysA + num_sysB
psi1 = hrf.random_statevector(num_sysA)
psi2 = hrf.random_statevector(num_sysB)
sv = np.kron(psi1, psi2).reshape(2**(num_qubits),1)
rho = sv @ sv.T.conj()

LN = hrf.logarithmic_negativity(rho, num_sysA, num_qubits)
print("Logarithmic Negativity (exact) =", LN)

Logarithmic Negativity (exact) = 4.4847971053408774e-15


For the Bell state $\left|\Phi^{+}\right\rangle = (|00\rangle+|11\rangle)/\sqrt{2}$, one can verify $E_{\text{N}}(\left|\Phi^{+}\right\rangle\langle \Phi^{+}|)=1$.

In [4]:
import numpy as np
import hadamard_random_forest as hrf

num_sysA = 1
num_sysB = 1
num_qubits = num_sysA + num_sysB
sv = np.array([[1,0,0,1]]/np.sqrt(2)).reshape([4,1])
rho = sv @ sv.T.conj()

LN = hrf.logarithmic_negativity(rho, num_sysA, num_qubits)
print("Logarithmic Negativity (exact) =", LN)

Logarithmic Negativity (exact) = 0.9999999999999997


### Use HRF to estimate negativity

In [8]:
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime.fake_provider import FakeFez

backend_sim = AerSimulator.from_backend(FakeFez())

In [10]:
import dill as pickle

# Load pre-generated parameters
with open('params.pkl', 'rb') as f:
    loaded_param = pickle.load(f)

In [19]:
import math
from qiskit.circuit.library import RealAmplitudes
from qiskit.quantum_info import Statevector, state_fidelity, DensityMatrix
import hadamard_random_forest as hrf

shots = 100000
num_layers = 4
num_trees = 111

# Fix random seed
hrf.fix_random_seed(999)

for num_qubits in range(2, 9, 2):
    print(f"=================================== Simulate {num_qubits}-qubit =========================================")
    ansatz = RealAmplitudes(num_qubits, entanglement='pairwise', reps=num_layers, insert_barriers=True, skip_final_rotation_layer=True)
    params = loaded_param['parameter'][(num_qubits, num_layers)][0] 
    list_circuits = hrf.get_circuits(num_qubits, ansatz)

    # Collect samples
    samples = hrf.get_samples_noisy(num_qubits, list_circuits, shots, params, backend_sim, error_mitigation=True)

    # Reconstruct the satatevector
    statevector_approx = hrf.get_statevector(num_qubits, num_trees, samples, save_tree=False, show_tree=False)
    rho_approx = DensityMatrix(statevector_approx)

    # Evaluate circuit and recover the exact statevector
    ansatz.assign_parameters(params, inplace=True)
    rho = DensityMatrix(ansatz)

    # Calculate Stabilizer Entropy
    LN = hrf.logarithmic_negativity(rho, num_sysA=math.ceil(num_qubits / 2), num_qubits=num_qubits)
    LN_approx = hrf.logarithmic_negativity(rho_approx, num_sysA=math.ceil(num_qubits / 2), num_qubits=num_qubits)
    print("Logarithmic Negativity (exact) =", LN)
    print("Logarithmic Negativity (HRF) =", LN_approx)

=================================== Simulate 2-qubit =========================================
Logarithmic Negativity (exact) = 0.4607085373250065
Logarithmic Negativity (HRF) = 0.5037943745848781
=================================== Simulate 4-qubit =========================================
Logarithmic Negativity (exact) = 0.844756958281152
Logarithmic Negativity (HRF) = 0.8540890214338566
=================================== Simulate 6-qubit =========================================
Logarithmic Negativity (exact) = 1.6779210392164574
Logarithmic Negativity (HRF) = 1.6878049433906073
=================================== Simulate 8-qubit =========================================
Logarithmic Negativity (exact) = 2.297051063765075
Logarithmic Negativity (HRF) = 2.370611767843103


## Reference

[1] [Życzkowski, Karol, et al. "Volume of the set of separable states." Physical Review A 58.2 (1998): 883.](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.58.883)

---

## Software info

In [7]:
import qiskit
import qiskit_aer
import qiskit_ibm_runtime
import mthree

print(f'Qiskit: {qiskit.__version__}')
print(f'Qiskit Aer: {qiskit_aer.__version__}')
print(f'Qiskit IBM Runtime: {qiskit_ibm_runtime.__version__}')
print(f'Mthree error mitigation: {mthree.__version__}')

Qiskit: 2.0.0
Qiskit Aer: 0.17.0
Mthree error mitigation: 3.0.0
